In [1]:
import numpy
from scipy import interpolate
import sys
from optparse import OptionParser



In [2]:
def error(msg):
   """ write error message and quit
   """
   sys.stderr.write(msg + "\n")
   sys.exit(3)

In [3]:
def deriv(spec,h):
   """ calculate first derivative of function 'spec'
       using the central finite difference method up to 6th order,
       and for the first 3 and last 3 grid points the
       forward/backward finite difference method up to 2nd order.
       ...as used in f77-program and suggested by Zanazzi-Jona...
   """ 
   der_spec =[[i[0],0] for i in spec]

   length=len(spec)
   for i in range(3,length-3):
      der_spec[i][1]=(-1*spec[i-3][1]+9*spec[i-2][1]-45*spec[i-1][1]+45*spec[i+1][1]-9*spec[i+2][1]+1*spec[i+3][1])/(60*h)
   for i in range(0,3):
      der_spec[i][1]=(-11*spec[i][1]+18*spec[i+1][1]-9*spec[i+2][1]+2*spec[i+3][1])/(6*h)
   for i in range(length-3,length):
      der_spec[i][1]=(11*spec[i][1]-18*spec[i-1][1]+9*spec[i-2][1]-2*spec[i-3][1])/(6*h)

   return der_spec

In [4]:
def get_range(tspec,espec,w_incr,shift,start,stop):
   """ determine wavenumber range within the comparison between theoretical
       and experimental spectrum is performed (depends on the shift)
   """
   de1=start+shift-espec[0][0]
   if (de1 >= 0 ):
      de1=int((start+shift-espec[0][0])/w_incr+0.00001)
      enstart=de1
      tnstart=int((start-tspec[0][0])/w_incr+0.00001)
   else:
      de1=int((start+shift-espec[0][0])/w_incr-0.00001)
      enstart=0
      tnstart=int((start-tspec[0][0])/w_incr-de1+0.00001)
   de2=stop+shift-espec[-1][0]
   if (de2 <= 0 ):
      de2=int((stop+shift-espec[-1][0])/w_incr-0.00001)
      enstop=len(espec)+de2
      tnstop=len(tspec)+int((stop-tspec[-1][0])/w_incr-0.00001) 
   else:
      de2=int((stop+shift-espec[-1][0])/w_incr+0.00001)
      enstop=len(espec)
      tnstop=len(tspec)+int((stop-tspec[-1][0])/w_incr-de2-0.00001)
   return tnstart, tnstop, enstart, enstop
 
